In [5]:
import os
import requests

from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

# os.environ: .streamlit/secrets.toml 파일에 넣어주면 꺼내서 사용함
# 2025.1.22. 실행 결과 위 파일에서 가져오지 않음. .env 파일에서 가져옴.
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
# print(alpha_vantage_api_key)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


# Symbol
class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearchTool"
    description: Type[
        str
    ] = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        from duckduckgo_search import DDGS

        ddg = DDGS().text(query)
        return ddg


# class CompanyOverviewArgsSchema(BaseModel):
#     symbol: str = Field(
#         description="Stock symbol of the company.Example: AAPL,TSLA",
#     )


# # Overview: 개요
# class CompanyOverviewTool(BaseTool):
#     name: Type[str] = "CompanyOverview"
#     description: Type[
#         str
#     ] = """
#     Use this to get an overview of the financials of the company.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

#     def _run(self, symbol):
#         r = requests.get(
#             f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         )
#         return r.json()


# # Income Statement: 손익계산서
# class CompanyIncomeStatementTool(BaseTool):
#     name: Type[str] = "CompanyIncomeStatement"
#     description: Type[
#         str
#     ] = """
#     Use this to get the income statement of a company.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

#     def _run(self, symbol):
#         r = requests.get(
#             f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         )
#         return r.json()["annualReports"]


# # 주간 성과 보고
# class CompanyStockPerformanceTool(BaseTool):
#     name: Type[str] = "CompanyStockPerformance"
#     description: Type[
#         str
#     ] = """
#     Use this to get the weekly performance of a company stock.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

#     def _run(self, symbol):
#         r = requests.get(
#             f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         )
#         return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    # llm이 모든 tool을 반드시 사용하는 것은 아님. 더 많은 정보가 필요하다고 생각될 때 사용함.
    tools=[
        StockMarketSymbolSearchTool(),
        # CompanyOverviewTool(),
        # CompanyIncomeStatementTool(),
        # CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


[{'title': 'Connect, protect, and build everywhere | Cloudflare', 'href': 'https://www.cloudflare.com/', 'body': "Cloudflare's connectivity cloud helps you improve security, consolidate to reduce costs, and move faster than ever. Watch webinar. Global leaders, including 30% of the Fortune 1000, rely on Cloudflare. See how leading enterprises regain control with Cloudflare."}, {'title': 'What is Cloudflare? | Cloudflare', 'href': 'https://www.cloudflare.com/learning/what-is-cloudflare/', 'body': "Cloudflare is one of the world's largest networks. Today, businesses, non-profits, bloggers, and anyone with an Internet presence boast faster, more secure websites and apps thanks to Cloudflare. Millions of Internet properties are on Cloudflare, and our network is growing by tens of thousands each day."}, {'title': 'Why Cloudflare? | Cloudflare', 'href': 'https://www.cloudflare.com

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "To analyze Cloudflare as a potential investment, we need to look at its stock symbol, financials, income statements, and stock performance. \n\n### Stock Symbol\nCloudflare trades under the ticker symbol **NET** on the New York Stock Exchange (NYSE).\n\n### Company Overview\nCloudflare, Inc. is a company that provides a range of services including content delivery network services, cloud cybersecurity, DDoS mitigation, and more. It is known for its extensive global network and is used by millions of websites.\n\n### Financial Information\nTo provide a comprehensive analysis, we would typically look at the following financial metrics:\n\n1. **Revenue Growth**: Look at the year-over-year revenue growth to assess how quickly the company is expanding.\n2. **Profitability**: Examine net income, 